In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Punto 1

In [83]:
np.random.seed(42)
#tomamos 5 documentos al azar
random_idxs = np.random.choice(len(newsgroups_train.data), 5)
print(random_idxs)

[ 7270   860  5390  5191 11284]


In [118]:
for i in random_idxs:
  doc = newsgroups_train.data[i]
  doc_vec = X_train[i]
  print(f"📁 Document id={i} label: {newsgroups_train.target_names[y_train[i]]}")
  print(doc[:500])
  cossim = cosine_similarity(doc_vec, X_train)[0]
  most_sim = np.argsort(cossim)[::-1][1:6]
  for j in most_sim:
    print(f"\t📝 Document id={j} similar to id={i}, with label: {newsgroups_train.target_names[y_train[j]]}")
    print(f'id={j}: {newsgroups_train.target_names[y_train[j]]}')
    related_doc = newsgroups_train.data[j]
    print(related_doc[:500])
    print("\n")
  print("\n")

📁 Document id=7270 label: rec.sport.hockey

I think Murray has done a great job.  He's picked up Ciccarelli,
Sheppard, Ysebaert, Howe, Coffey, and Riendeau (plus some depth players) 
without giving up anything the Wings needed or any of his top prospects.
All of this in three years.  Has anyone done better?

The year before he took over, the Wings didn't even make the playoffs.
There was about a year and a half during Demers' stint that the Wings
did OK, but that was due to Demers' motavational skills and clutch
and grab style.  They did
	📝 Document id=1880 similar to id=7270, with label: rec.sport.hockey
id=1880: rec.sport.hockey

Devallano went earlier and more extensively to the Russian strategy
than anyone else...and was the first GM to "waste" high draft choices
on young Russians...Devallano would still be GM but he succombed to
Demers pleading to make the Oates-Federko et al trade...which is the
deal that sealed his fate.

Murray has made some decent trades...no doubt...but these

📁 **Document id=7270** label: rec.sport.hockey

### 🔗 Relacionados:
- 📝 **id=1880** label: rec.sport.hockey  
- 📝 **id=1292** label: talk.politics.mideast  
- 📝 **id=6437** label: talk.politics.mideast  
- 📝 **id=9623** label: talk.politics.mideast  
- 📝 **id=7286** label: talk.politics.mideast  

---

📁 **Document id=860** label: talk.politics.mideast  

### 🔗 Relacionados:
- 📝 **id=7714** label: comp.windows.x  
- 📝 **id=2551** label: sci.crypt  
- 📝 **id=6920** label: talk.politics.mideast  
- 📝 **id=5825** label: talk.politics.mideast  
- 📝 **id=7956** label: talk.politics.misc  

---

📁 **Document id=5390** label: alt.atheism  

### 🔗 Relacionados:
- 📝 **id=20** label: alt.atheism  
- 📝 **id=6376** label: alt.atheism  
- 📝 **id=1137** label: alt.atheism  
- 📝 **id=7491** label: alt.atheism  
- 📝 **id=10779** label: alt.atheism  

---

📁 **Document id=5191** label: rec.autos  


### 🔗 Relacionados:
- 📝 **id=8714** label: soc.religion.christian  
- 📝 **id=10779** label: alt.atheism  
- 📝 **id=6437** label: talk.politics.mideast  
- 📝 **id=5988** label: talk.politics.guns  
- 📝 **id=9623** label: talk.politics.mideast  

---

📁 **Document id=11284** label: comp.sys.mac.hardware  


### 🔗 Relacionados:
- 📝 **id=7793** label: comp.sys.mac.hardware  
- 📝 **id=4651** label: sci.electronics  
- 📝 **id=9892** label: comp.sys.mac.hardware  
- 📝 **id=3080** label: comp.sys.ibm.pc.hardware  
- 📝 **id=1079** label: rec.motorcycles  


Claramente hay categorías que no tienen nada que ver entre sí, por ejemplo en el primer documento en los relacionados se encuentra mideast, que no tiene nada en común con hockey.
Como en ese ejemplo, pasa con los otros, es decir que no lo podemos usar para asociación así como así. Para que este tipo de análisis sea correcto hay que agregar un preprocesamiento, como por ejemplo filtrar las stopwords.